# Simulación de una avenida con paso peatonal
En esta simulación se representa una avenida con dos carriles y un paso peatonal. El tráfico se controla con dos semáforos que detienen a los autos en el paso peatonal.

## Librerías para la simulación

In [1]:
import agentpy as ap
import numpy as np
import matplotlib.pyplot as plt
import math
import IPython
import json

## Definición del agente semáforo

In [2]:
class Semaphore(ap.Agent):
    """ 
        Esta clase define a un semáforo.
    """

    def setup(self):
        """ Este método se utiliza para inicializar al semáforo. """
        self.step_time = 0.1         # Tiempo que dura cada paso de la simulación

        self.direction = [0, 1]      # Dirección a la que apunta el semáforo

        self.state = 0               # Estado del semáforo 0 = verde, 1 = amarillo, 2 = rojo
        self.state_time = 0          # Tiempo que ha durado el semáforo en el estado actual

        self.green_duration = 5.0     # Tiempo que dura el semáforo en verde
        self.yellow_duration = 0.5     # Tiempo que dura el semáforo en amarillo
        self.red_duration = 4.5       # Tiempo que dura el semáforo en rojo
        

    def update(self):
        """ Este método actualiza el estado del semáforo. """
        self.state_time += self.step_time

        if self.state == 0:
            # Caso en el que el semáforo está en verde
            if self.state_time >= self.green_duration:
                self.state = 1
                self.state_time = 0
        elif self.state == 1:
            # Caso en el que el semáforo está en amarillo
            if self.state_time >= self.yellow_duration:
                self.state = 2
                self.state_time = 0
        elif self.state == 2:
            # Caso en el que el semáforo está en rojo
            if self.state_time >= self.red_duration:
                self.state = 0
                self.state_time = 0

    def set_green(self):
        """ Este método forza el semáforo a estar en verde. """        
        self.state = 0
        self.state_time = 0

    def set_yellow(self):
        """ Este método forza el semáforo a estar en amarillo. """        
        self.state = 1
        self.state_time = 0

    def set_red(self):
        """ Este método forza el semáforo a estar en rojo. """        
        self.state = 2
        self.state_time = 0


## Definición del agente auto

In [3]:
class Car(ap.Agent):
    """ 
        Esta clase define a un auto.
    """

    def setup(self):
        """ Este método se utiliza para inicializar un robot limpiador. """
        self.step_time = 0.1         # Tiempo que dura cada paso de la simulación

        self.direction = [1, 0]      # Dirección a la que viaja el auto
        self.speed = 0.0             # Velocidad en metros por segundo
        self.max_speed = 0.4         # Máxima velocidad en metros por segundo
        self.state = 1               # Car state: 1 = ok, 0 = dead
        self.x = 0
        self.z = 0
        
    def update_position(self):
        """ Este método se utiliza para inicializar la posición del auto. """

        # Verifica si el auto no ha chocado
        if self.state == 0:
            return

         # Actualiza la posición según la velocidad actual
        self.model.avenue.move_by(self, [self.speed*self.direction[0], self.speed*self.direction[1]])
        if(self.direction[1] == -1):
            self.x += self.speed*self.direction[0]
            self.z += self.speed*self.direction[1]
        elif(self.direction[1] == 1):
            self.x += self.speed*self.direction[0]
            self.z += self.speed*self.direction[1]
        
        

    def update_speed(self):
        
        """ Este método se utiliza para inicializar la velocidad del auto. """

        # Verifica si el auto no ha chocado
        if self.state == 0:
            return
        
        # Obten la distancia más pequeña a uno de los autos que vaya en la misma dirección        
        p = self.model.avenue.positions[self]

        min_car_distance = 1000000
        for car in self.model.cars:
            if car != self:
                # Verifica si el carro va en la misma dirección
                dot_p1 = self.direction[0]*car.direction[0] + self.direction[1]*car.direction[1]                
                
                # Verifica si el carro está atrás o adelante
                p2 = self.model.avenue.positions[car]
                dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]

                if dot_p1 > 0 and dot_p2 > 0:                     
                    d = math.sqrt((p[0]-p2[0])**2 + (p[1]-p2[1])**2)                    
                    
                    if min_car_distance > d:
                        min_car_distance = d
       
        # Obten la distancia al próximo semáforo
        min_semaphore_distance = 1000000
        semaphore_state = 0
        for semaphore in self.model.semaphores:
            # Verifica si el semáforo apunta hacia el vehículo
            dot_p1 = semaphore.direction[0]*self.direction[0] + semaphore.direction[1]*self.direction[1]
            
            # Verifica si el semáforo está adelante o atrás del vehículo
            p2 = self.model.avenue.positions[semaphore]
            dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]

            if dot_p1 < 0 and dot_p2 > 0:                            
                d = math.sqrt((p[0]-p2[0])**2 + (p[1]-p2[1])**2)  
                
                if min_semaphore_distance > d:
                    min_semaphore_distance = d
                    semaphore_state = semaphore.state
        
        # Actualiza la velocidad del auto
        print(min_semaphore_distance)
        if min_car_distance < 0.2:
            self.speed = 0
            self.state = 1

        elif min_car_distance < 2:
              self.speed = np.maximum(self.speed - 20*self.step_time, 0)

        elif min_car_distance < 5:
              self.speed = np.maximum(self.speed - 8*self.step_time, 0)
                
        elif min_semaphore_distance < 5 and semaphore_state == 1:
            self.speed = np.minimum(self.speed + 0.5*self.step_time, self.max_speed)

        elif min_semaphore_distance < 10 and semaphore_state == 1:
            self.speed = np.maximum(self.speed - 2*self.step_time, 0)
            
        elif min_semaphore_distance < 5 and semaphore_state == 2:
            self.speed = np.maximum(self.speed - 8*self.step_time, 0)

        else:
            self.speed = np.minimum(self.speed + 0.5*self.step_time, self.max_speed)
                    


## Definición del modelo de la avenida

In [4]:
class AvenueModel(ap.Model):
    """ Esta clase define un modelo para una avenida simple con semáforo peatonal. """

    def setup(self):
        """ Este método se utiliza para inicializar la avenida con varios autos y semáforos. """
        # Inicializa los agentes los autos y los semáforos        
        self.cars = ap.AgentList(self, self.p.cars, Car)
        self.cars.step_time =  self.p.step_time
        
        c_north = int(self.p.cars/2)
        c_south = self.p.cars - c_north

        for k in range(c_north):
            self.cars[k].direction = [0,1]

        for k in range(c_south):
            self.cars[k+c_north].direction = [0,-1]

        self.semaphores = ap.AgentList(self,2, Semaphore)
        self.semaphores.step_time =  self.p.step_time
        self.semaphores.green_duration = self.p.green
        self.semaphores.yellow_duration = self.p.yellow
        self.semaphores.red_duration = self.p.red
        self.semaphores[0].direction = [0, 1]
        self.semaphores[1].direction = [0, -1]

        # Inicializa el entorno
        self.avenue = ap.Space(self, shape=[60, self.p.size], torus = True)
                
        # Agrega los semáforos al entorno
        self.avenue.add_agents(self.semaphores, random=True)
        self.avenue.move_to(self.semaphores[0], [0, self.p.size*0.5 + 5])
        self.avenue.move_to(self.semaphores[1], [10, self.p.size*0.5 - 5])

        # Agrega los autos al entorno
        self.avenue.add_agents(self.cars, random=True)
        for k in range(c_north):
            self.avenue.move_to(self.cars[k], [10, 10*(k+1)])
            self.cars[k].x = 40
            self.cars[k].z = 10*(k+1)
        
        for k in range(c_south):
            self.avenue.move_to(self.cars[k+c_north], [0, self.p.size - (k+1)*10])
            self.cars[k+c_north].x = 20
            self.cars[k+c_north].z = self.p.size - (k+1)*10

        self.actualStep = 0
        self.data = {}
        self.data['frames'] = []
        self.data['frames'].append({
            'frame': self.actualStep,
            'car': [],
            'trafficLight': []
        })
        
        id = 0
        
        for car in self.cars:
            self.data['frames'][self.actualStep]['car'].append({
                'id': id,
                'x': (car.x - 30)/2,
                'z': car.z - self.p.size/2,
                'dir': car.direction[1]
            })
            id += 1
        for semaphore in self.semaphores:
            if(semaphore.direction[1] == 1):
                self.data['frames'][self.actualStep]['trafficLight'].append({
                    'id': id,
                    'x': 0,
                    'z': self.p.size*0.5 + 5 - self.p.size/2,
                    'dir': semaphore.direction[1],
                    'state': semaphore.state
                })
            elif(semaphore.direction[1] == -1):
                self.data['frames'][self.actualStep]['trafficLight'].append({
                    'id': id,
                    'x': 0,
                    'z': self.p.size*0.5 - 5 - self.p.size/2,
                    'dir': semaphore.direction[1],
                    'state': semaphore.state
                })
            id += 1
            
    def step(self):
        """ Este método se invoca para actualizar el estado de la avenida. """        
        self.semaphores.update()
        self.cars.update_position()
        self.cars.update_speed()
        
        self.actualStep +=1
        
        self.data['frames'].append({
            'frame': self.actualStep,
            'car': [],
            'trafficLight': []
        })
        
        id = 0
        
        for car in self.cars:
            self.data['frames'][self.actualStep]['car'].append({
                'id': id,
                'x': (car.x - 30)/2,
                'z': car.z - self.p.size/2,
                'dir': car.direction[1]
            })
            id += 1
        for semaphore in self.semaphores:
            if(semaphore.direction[1] == 1):
                self.data['frames'][self.actualStep]['trafficLight'].append({
                    'id': id,
                    'x': 0,
                    'z': self.p.size*0.5 + 5 - self.p.size/2,
                    'dir': semaphore.direction[1],
                    'state': semaphore.state
                })
            elif(semaphore.direction[1] == -1):
                self.data['frames'][self.actualStep]['trafficLight'].append({
                    'id': id,
                    'x': 0,
                    'z': self.p.size*0.5 - 5 - self.p.size/2,
                    'dir': semaphore.direction[1],
                    'state': semaphore.state
                })
            id += 1

    def end(self):
        jsonObj = json.dumps(self.data, indent = 4)
        with open('data.json', 'w') as outfile:
            outfile.write(jsonObj)
        

## Funciones para visualización

In [5]:
def animation_plot_single(m, ax):    
    ax.set_title(f"Avenida t={m.t*m.p.step_time:.2f}")
    
    colors = ["green", "yellow", "red"]
    
    pos_s1 = m.avenue.positions[m.semaphores[0]]    
    ax.scatter(*pos_s1, s=20, c=colors[m.semaphores[0].state])
    
    pos_s2 = m.avenue.positions[m.semaphores[1]]    
    ax.scatter(*pos_s2, s=20, c=colors[m.semaphores[1].state])
    
    ax.set_xlim(0, m.avenue.shape[0])
    ax.set_ylim(0, m.avenue.shape[1])    
    
    for car in m.cars:
        pos_c = m.avenue.positions[car]    
        ax.scatter(*pos_c, s=20, c="black")
    
    ax.set_axis_off()
    ax.set_aspect('equal', 'box')
        
def animation_plot(m, p):    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=20)) 

## Parámetros de la simulación

In [12]:
parameters = {
    'step_time': 0.1,    # Procentaje de área cubierta por árboles
    'size': 200,        # Tamaño en metros de la avenida
    'green': 5,          # Duración de la luz verde
    'yellow': 0.5,         # Duración de la luz amarilla
    'red': 5,           # Duración de la luz roja
    'cars': 8,          # Número de autos en la simulación
    'steps': 600,       # Número de pasos de la simulación
}

## Simulación de una corrida

In [13]:
model = AvenueModel(parameters)
results = model.run()

85.0
75.0
65.0
55.0
85.0
75.0
65.0
55.0
Completed: 1 steps84.95
74.95
64.95
54.95
84.94999999999999
74.94999999999999
64.94999999999999
54.94999999999999
Completed: 2 steps84.85
74.85
64.85
54.85
84.85
74.85
64.85
54.849999999999994
Completed: 3 steps84.7
74.7
64.7
54.7
84.69999999999999
74.69999999999999
64.69999999999999
54.69999999999999
Completed: 4 steps84.5
74.5
64.5
54.5
84.5
74.5
64.5
54.5
Completed: 5 steps84.25
74.25
64.25
54.25
84.25
74.25
64.25
54.25
Completed: 6 steps83.95
73.95
63.95
53.95
83.94999999999999
73.94999999999999
63.94999999999999
53.94999999999999
Completed: 7 steps83.6
73.6
63.599999999999994
53.6
83.6
73.6
63.599999999999994
53.599999999999994
Completed: 8 steps83.2
73.2
63.2
53.2
83.19999999999999
73.19999999999999
63.19999999999999
53.19999999999999
Completed: 9 steps82.8
72.8
62.8
52.800000000000004
82.79999999999998
72.79999999999998
62.79999999999998
52.79999999999998
Completed: 10 steps82.4
72.4
62.4
52.400000000000006
82.39999999999998
72.39999999999

1000000
Completed: 157 steps23.600000000000065
13.599999999999895
3.5999999999997243
1000000
23.59999999999914
13.599999999999142
3.5999999999991417
1000000
Completed: 158 steps23.20000000000006
13.19999999999989
3.1999999999997186
1000000
23.199999999999136
13.199999999999136
3.199999999999136
1000000
Completed: 159 steps22.800000000000054
12.799999999999883
2.799999999999713
1000000
22.79999999999913
12.79999999999913
2.7999999999991303
1000000
Completed: 160 steps22.40000000000005
12.399999999999878
2.3999999999997073
1000000
22.399999999999125
12.399999999999125
2.3999999999991246
1000000
Completed: 161 steps22.000000000000043
11.999999999999872
1.9999999999997016
1000000
21.99999999999912
11.999999999999119
1.999999999999119
1000000
Completed: 162 steps21.600000000000037
11.599999999999866
1.5999999999996959
1000000
21.599999999999113
11.599999999999113
1.5999999999991132
1000000
Completed: 163 steps21.20000000000003
11.19999999999986
1.5999999999996959
1000000
21.199999999999108


1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 303 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 304 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 305 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 306 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 307 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 308 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 309 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 310 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 311 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 312 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 313 steps1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
Completed: 314 steps

1000000
1000000
1000000
82.79999999999865
Completed: 435 steps1000000
1000000
1000000
82.39999999999816
1000000
1000000
1000000
82.39999999999864
Completed: 436 steps1000000
1000000
1000000
81.99999999999815
1000000
1000000
1000000
81.99999999999864
Completed: 437 steps1000000
1000000
1000000
81.59999999999816
1000000
1000000
1000000
81.59999999999863
Completed: 438 steps1000000
1000000
1000000
81.19999999999816
1000000
1000000
1000000
81.19999999999862
Completed: 439 steps1000000
1000000
1000000
80.79999999999815
1000000
1000000
1000000
80.79999999999862
Completed: 440 steps1000000
1000000
1000000
80.39999999999816
1000000
1000000
1000000
80.39999999999861
Completed: 441 steps1000000
1000000
1000000
79.99999999999815
1000000
1000000
1000000
79.99999999999861
Completed: 442 steps1000000
1000000
1000000
79.59999999999815
1000000
1000000
1000000
79.5999999999986
Completed: 443 steps1000000
1000000
1000000
79.19999999999816
1000000
1000000
1000000
79.1999999999986
Completed: 444 steps1000

Completed: 600 steps
Run time: 0:00:00.871107
Simulation finished


## Visualización de una corrida

In [ ]:
animation_plot(AvenueModel, parameters)